In [54]:
from sympy import *
import numpy as np

# Fixed values

W_frame = 9.2 * 1e-3 # kg
W_FC_ESC = 34 * 1e-3 # kg
W_RX = 0.55 * 1e-3 # kg
W_batt_sup = 2.24 * 1e-3 # kg
c = 0.009525 # m (blade)
rho = 1.225 # kg/m3
hub = 0.01 # m; Assumed hub diameter of the propeller

W_fixed = W_frame + W_FC_ESC + W_RX + W_batt_sup

############### CHANGE THE FOLLOWING ###############

# Variable Quantities -- Propeller
prop_count = 3
prop_diameter_inch = 3
prop_pitch_inch = 1.6
prop_weight = 1.24

# Variable Quantities -- Batteries
batt_weight = 74
batt_mAh = 850

# Variable Quantities -- Motor
motor_weight = 5.9

####################################################

# Unit Conversion
motor_weight *= 1e-3 # kg
prop_weight *= 1e-3 # kg
batt_weight *= 1e-3 # kg
batt_Ah = batt_mAh * 1e-3 # Ah
prop_pitch = prop_pitch_inch / 39.37
prop_diameter = prop_diameter_inch / 39.37

# Prelim Calculation
motor_weight *= 4
prop_weight *= 4

# Alpha
prop_radius = prop_diameter / 2 - (hub/2)
alpha = atan(prop_pitch / (pi*(prop_diameter))) # Angle of Attack of prop cross section

# Set up linear interpolation for RPM & Current
thr = np.array([30, 40, 50, 60, 70, 80, 90, 100])

############### CHANGE THE FOLLOWING IF MOTOR CHANGED ###############

if prop_count == 3:
    rpm = np.array([11800, 14700, 17000, 19000, 21000, 22400, 24000, 25500])
    am = np.array([0.52, 0.90, 1.32, 1.92, 2.60, 3.45, 4.30, 5.17])
elif prop_count == 2:
    rpm = np.array([12400, 15400, 18000, 20500, 23100, 25600, 27200, 29700])
    am = np.array([0.4, 0.8, 1.2, 1.7, 2.4, 3.2, 4.1, 5.2])
    
#####################################################################

# Find RPM
m = motor_weight + prop_weight + batt_weight + W_fixed
V, theta = symbols('V, theta', real = True)
L = m * 9.81
Ldepth = (L/4) / 3 / prop_radius  # Lift per unit depth
integral = integrate(2*alpha*V*((1+cos(theta))/sin(theta))*(c/2)*sin(theta),(theta,0,pi))  # Circulation
eqn = Eq(Ldepth, rho*V*integral)  # Solve for V_inf
omega = solve(eqn)[1] / (prop_radius/2)
RPM = (omega * 60 / (2*pi)).evalf()

throttle_hover = np.interp(RPM, rpm, thr)
current_hover = np.interp(RPM, rpm, am)
flight_time = (batt_Ah / (current_hover * 4))*60

flight_time_max_thr = (batt_Ah / (am[-1]*4))*60

print(f'Drone weight: {round(m,3)} kg')
print(f'Batt Spec: Weight = {batt_weight} kg, Capacity = {batt_mAh} mAh')
print(f'Prop Spec: Diameter = {prop_diameter_inch} inch, Pitch = {prop_pitch_inch} inch, Blade count = {prop_count}')
print(f'Hover Lift: {round(L,3)} N')
print(f'Hover Lift: {round(L/4,3)} N per motor')
print(f'Hover RPM: {round(RPM)} RPM per motor')
print(f'Hover throttle: {round(throttle_hover,1)} %')
print(f'Hover endurance: {round(flight_time,1)} minutes')
print(f'Max throttle endurance: {round(flight_time_max_thr,1)} minutes')

Drone weight: 0.149 kg
Batt Spec: Weight = 0.074 kg, Capacity = 850 mAh
Prop Spec: Diameter = 3 inch, Pitch = 1.6 inch, Blade count = 3
Hover Lift: 1.457 N
Hover Lift: 0.364 N per motor
Hover RPM: 14077 RPM per motor
Hover throttle: 37.9 %
Hover endurance: 15.6 minutes
Max throttle endurance: 2.5 minutes
